# Example Usage

Start by getting a dummy file

In [9]:
from ftag import get_dummy_file
f = get_dummy_file()
jets = f['jets']

### Cuts

The `Cuts` class provides an interface for applying selections to structured nummpy arrays loaded from HDF5 files.
To take a look, first import the `Cuts`:


In [2]:
from ftag import Cuts

Instances of `Cuts` can be defined from lists of strings or tuples of strings and values. For example

In [3]:
kinematic_cuts = Cuts.from_list(["pt > 20e3", "abs_eta < 2.5"])
flavour_cuts = Cuts.from_list([("HadronConeExclTruthLabelID", "==", 5)])

It's easy to combine cuts

In [4]:
combined_cuts = kinematic_cuts + flavour_cuts

And then apply them to a a structured array with 

In [5]:
idx, selected_jets = combined_cuts(jets)

Both the selected indices and the selected jets are returned. The indices can be used to reapply the same selection on another array (e.g. tracks). The return values `idx` and `values` can also be accessed directly

In [9]:
idx = combined_cuts(jets).idx
selected_jets = combined_cuts(jets).values

### Flavours

A list of flavours is provided.

In [3]:
from ftag import Flavours
Flavours.bjets

Flavour(name='bjets', label='$b$-jets', cuts=['HadronConeExclTruthLabelID == 5'], colour='#1f77b4')

`dict` like access is also supported:

In [5]:
Flavours["qcd"]

Flavour(name='qcd', label='QCD', cuts=['R10TruthLabel_R22v1 == 10'], colour='#38761D')

As you can see from the output, each flavour has a `name`, a `label` and `colour` (used for plotting), and a `Cuts` instance, which can be used to select jets of the given flavour.
For example:

In [12]:
bjets = Flavours.bjets.cuts(jets).values

Probability names are also accessible using `.px`:

In [14]:
[f.px for f in Flavours]

['pb', 'pc', 'pu', 'ptau', 'phbb', 'phcc', 'ptop', 'pqcd']

### H5Reader

Allows for batched reading from one or more h5 files. For example


from ftag import H5Reader

# read from a single file, globbing is supported
reader = H5Reader("path/to/file/*.h5", batch_size=100)
stream = reader.stream({"jets": ["pt", "eta"]}, num_jets=1000)

# will read 10 batches of 100 jets
for batch in stream:
    print(len(batch["jets"]))


Reading tracks is also supported

In [ ]:
stream = reader.stream({"jets": ["pt", "eta"]}, num_jets=1000)



### H5Writer
